In [1]:
import wandb

In [2]:
# API Key a5b5beccacb40d16d32ca7910e86bcc9ce0d5fec
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: juliendelaunay35000 (juliendelaunay). Use `wandb login --relogin` to force relogin


True

In [3]:
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

acc,▁▆▅▆██▇▇
loss,█▄▆▃▅▁▁▂
acc,0.69128
loss,0.26472


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

hate_speech = pd.read_csv("./hate_speech.csv")
hate_speech.drop(["Unnamed: 0"], axis=1, inplace=True)
hate_speech["label"].replace({"noHate":0, "hate":1, "relation":2}, inplace=True)
hate_speech.drop(hate_speech[hate_speech['label'] == "idk/skip"].index, inplace = True)

X_train, X_test, y_train, y_test = train_test_split(hate_speech["text"], hate_speech["label"], test_size=0.2, random_state=42)

C:\Users\User\AppData\Local\Temp\ipykernel_8916\3284763887.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\User\AppData\Local\Temp\ipykernel_8916\3284763887.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TextClassificationPipeline

tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")

def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True)["input_ids"]
    
X_train_vect = X_train.map(tokenize_function)
X_test_vect = X_test.map(tokenize_function)


c:\Users\User\miniconda3\envs\wb\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [6]:
from sklearn.ensemble import GradientBoostingClassifier
y_train, y_test = y_train.astype(int), y_test.astype(int)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                max_depth=1, random_state=0)
clf.fit(X_train_vect.to_list(), y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [7]:
clf.score(X_test_vect.to_list(), y_test)

0.8698850574712643

In [8]:
from sklearn.metrics import accuracy_score

wandb.init(project="visualize-sklearn")

y_pred = clf.predict(X_test_vect.to_list())
accuracy = accuracy_score(y_test, y_pred)

# If logging metrics over time, then use wandb.log
wandb.log({"accuracy": accuracy})

# OR to log a final metric at the end of training you can also use wandb.summary
wandb.summary["accuracy"] = accuracy

In [11]:
# Visualize single plot
labels = ["noHate", "hate", "relation"]
y_probas = clf.predict_proba(X_test_vect.to_list())
print(set(y_test))
print(set(y_pred))
y_test_wandb = []
for y in y_test:
    y_test_wandb.append(int(y))
y_pred_wandb = []
for y in y_pred:
    y_pred_wandb.append(int(y))
print(y_test_wandb)
print(y_pred_wandb)
wandb.sklearn.plot_confusion_matrix(y_test_wandb, y_pred_wandb, labels)

{0, 1, 2}
{0, 1, 2}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [12]:
# Visualize all classifier plots
wandb.sklearn.plot_classifier(
    clf,
    X_train_vect.to_list(),
    X_test_vect.to_list(),
    y_train,
    y_test,
    y_pred,
    y_probas,
    labels,
    model_name="GB",
    feature_names=None,
)

# All regression plots
#wandb.sklearn.plot_regressor(reg, X_train, X_test, y_train, y_test, model_name="Ridge")

# All clustering plots
#wandb.sklearn.plot_clusterer(kmeans, X_train, cluster_labels, labels=None, model_name="KMeans")

wandb: 
wandb: Plotting GB.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: WARNING This function only supports binary classification at the moment and therefore expects labels to be binary. Skipping calibration curve.
wandb: Logged calibration curve.
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


In [13]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import pandas as pd
import wandb
wandb.init(project="sklearn")

# Load data
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = housing.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train model, get predictions
reg = Ridge()
reg.fit(X, y)
y_pred = reg.predict(X_test)

# Visualize all regression plots
wandb.sklearn.plot_regressor(reg, X_train, X_test, y_train, y_test, 'Ridge')

# Make individual plots
wandb.sklearn.plot_outlier_candidates(reg, X, y)

accuracy,▁
accuracy,0.86989


wandb: 
wandb: Plotting Ridge.
c:\Users\User\miniconda3\envs\wb\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
wandb: Logged summary metrics.
wandb: Logged learning curve.
c:\Users\User\miniconda3\envs\wb\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
wandb: WARNING using only the first 1000 datapoints to create chart outlier_candidates
wandb: Logged outlier candidates.
c:\Users\User\miniconda3\envs\wb\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
wandb: WARNING using only the first 100 datapoints to create chart residuals
wandb: WARNING using only the first 100 datapoints to create chart residuals
wandb: Logged residuals.
c:\Users\User\miniconda3\envs\wb\Lib\site-packages\sklearn\base.py:493:

In [14]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
categories = [
        "alt.atheism",
        "talk.religion.misc",
        "comp.graphics",
        "sci.space",
]
remove = ("headers", "footers", "quotes")
data_train = fetch_20newsgroups(
    subset="train", categories=categories, shuffle=True, random_state=42, remove=remove
)

data_test = fetch_20newsgroups(
    subset="test", categories=categories, shuffle=True, random_state=42, remove=remove
)
print("data loaded")

# order of labels in `target_names` can be different from `categories`
target_names = data_train.target_names
y_train, y_test = data_train.target, data_test.target

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words="english")
X_train = vectorizer.fit_transform(data_train.data)
X_test = vectorizer.transform(data_test.data)

reg = RidgeClassifier()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
labels = ["Atheism", "religion", "graphic", "space"]
wandb.sklearn.plot_confusion_matrix(y_test, y_pred, labels)

data loaded
